<a href="https://colab.research.google.com/github/Yanina-Kutovaya/GNN/blob/main/notebooks/Mock_PostGres_to_custom_loader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mock-тест для BitcoinGraphDataset


Тестовый класс TestBitcoinGraphDataset охватывает большинство аспектов работы класса BitcoinGraphDataset.

Цели тестов:

1. ```test_neighbor_sampling``` - Проверить семплирование окрестностей узлов
2. ```test_minimal_neighborhood``` -  Проверить обработку минимальных данных.
3. ```test_sampling_errors``` - Проверить обработку ошибок при семплировании.
4. ```test_sql_query_format``` - Убедиться, что SQL-запросы формируются корректно.
5. ```test_real_data_sampling``` - Проверить обработку реальных данных.
6. ```test_getitem_returns_correct_data``` - Проверить, что метод  ```__getitem__``` датасета корректно формирует граф с правильными тензорами признаков (```x```), индексов рёбер (```edge_index```), атрибутов рёбер (```edge_attr```) и меток классов (```y```).
7. ```test_getitem_returns_empty_graph_when_no_edges``` - Проверить обработку отсутствия рёбер.
8. ```test_len_returns_correct_value``` - Проверить корректность метода ```__len__```
9. ```test_model_compatibility```- Проверить, что данные из датасета можно передать в GCN-модель.








In [1]:
!pip install -q torch torch-geometric

In [2]:
import logging
import psycopg2
import pandas as pd
import torch
from torch_geometric.data import Data

import unittest
from unittest.mock import patch, MagicMock
import pandas as pd
from torch_geometric.nn import GCNConv

# Настройка логирования
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class BitcoinGraphDataset(torch.utils.data.Dataset):
    """Dataset для загрузки графа биткоина с семплированием окрестностей."""

    def __init__(self, db_config, batch_size=32, num_hops=2, num_neighbors=5):
        """Инициализация датасета.

        Args:
            db_config: Конфигурация БД
            batch_size: Размер батча
            num_hops: Количество шагов в семплировании
            num_neighbors: Количество соседей на шаг
        """
        self.db_config = db_config
        self.batch_size = batch_size
        self.num_hops = num_hops
        self.num_neighbors = num_neighbors
        self.conn = None
        self.cursor = None
        self.total_nodes = 0

        self._init_db_connection()
        self.total_nodes = self._get_total_nodes() or 0
        logger.info(f"Инициализирован датасет с {self.total_nodes} узлами.")

    def _init_db_connection(self):
        """Инициализация подключения к БД."""
        try:
            self.conn = psycopg2.connect(**self.db_config)
            if isinstance(self.conn, psycopg2.extensions.connection):
                self.cursor = self.conn.cursor()
                logger.info("Успешное подключение к базе данных.")
            else:
                raise TypeError("Подключение не является ожидаемым psycopg2 connection")
        except Exception as e:
            logger.error(f"Ошибка подключения к БД: {e}")
            self.conn = self.cursor = None

    def _get_total_nodes(self):
        """Получение общего количества узлов."""
        if not self.cursor or not isinstance(self.cursor, psycopg2.extensions.cursor):
            return 0
        try:
            query = "SELECT COUNT(*) FROM node_attributes"
            logger.debug(f"Выполняется SQL-запрос: {str(query)}")
            self.cursor.execute(query)
            result = self.cursor.fetchone()
            count = result[0] if result else 0
            logger.info(f"Найдено {count} узлов в таблице node_attributes.")
            return count
        except Exception as e:
            logger.error(f"Ошибка получения количества узлов: {e}")
            return 0

    def __len__(self):
        """Размер датасета в батчах."""
        return (self.total_nodes + self.batch_size - 1) // self.batch_size

    def __getitem__(self, idx):
        """Получение одного батча."""
        if not self._check_connection():
            logger.warning("Соединение с БД потеряно. Возвращается пустой граф.")
            return self._empty_data()
        try:
            nodes_df = self._load_batch_nodes(idx)
            if nodes_df.empty:
                logger.warning(f"Батч {idx} не содержит узлов.")
                return self._empty_data()

            all_aliases = self._get_neighborhood_aliases(nodes_df['alias'].tolist())
            if not all_aliases:
                logger.warning(f"Не удалось получить окрестности для батча {idx}.")
                return self._empty_data()

            all_nodes_df = self._load_node_attributes(all_aliases)
            edges_df = self._load_edges(all_aliases)

            logger.info(f"Сконструирован граф с {len(all_nodes_df)} узлами и {len(edges_df)} рёбрами.")
            return self._construct_graph_data(all_nodes_df, edges_df)
        except Exception as e:
            logger.error(f"Ошибка при получении батча {idx}: {e}")
            return self._empty_data()

    def _check_connection(self):
        """Проверка активности подключения."""
        return bool(self.conn and self.cursor and isinstance(self.conn, psycopg2.extensions.connection))

    def _load_batch_nodes(self, idx):
        """Загрузка батча узлов."""
        offset = idx * self.batch_size
        query = f"""
            SELECT alias, label, degree, total_received
            FROM node_attributes
            LIMIT {self.batch_size} OFFSET {offset}
        """
        logger.debug(f"Загрузка батча узлов: {str(query)}")
        try:
            df = pd.read_sql(query, self.conn)
            logger.info(f"Загружено {len(df)} узлов из батча {idx}.")
            return df
        except Exception as e:
            logger.error(f"Ошибка при чтении узлов: {e}")
            return pd.DataFrame()

    def _get_neighborhood_aliases(self, seed_aliases):
        """Получение окрестностей для узлов."""
        if not seed_aliases:
            logger.warning("Список алиасов пуст. Невозможно выполнить семплирование окрестностей.")
            return []

        query = f"""
            WITH RECURSIVE search_graph AS (
                SELECT alias, 0 AS hop
                FROM node_attributes
                WHERE alias IN ({','.join(map(str, seed_aliases))})
                UNION ALL
                SELECT next_alias, sg.hop + 1
                FROM (
                    SELECT
                        CASE WHEN a = sg.alias THEN b ELSE a END AS next_alias,
                        sg.alias AS current,
                        sg.hop,
                        ROW_NUMBER() OVER (PARTITION BY sg.alias ORDER BY random()) AS rn
                    FROM edge_attributes e
                    INNER JOIN search_graph sg ON e.a = sg.alias OR e.b = sg.alias
                    WHERE sg.hop < {self.num_hops}
                ) sub
                WHERE next_alias IS NOT NULL AND rn <= {self.num_neighbors}
            )
            SELECT DISTINCT alias FROM search_graph;
        """
        logger.debug(f"Семплирование окрестностей: {str(query)}")
        try:
            self.cursor.execute(query)
            aliases = [row[0] for row in self.cursor.fetchall()]
            logger.info(f"Найдено {len(aliases)} узлов в окрестности.")
            return aliases
        except Exception as e:
            logger.error(f"SQL Error in neighborhood sampling: {e}")
            return []

    def _load_node_attributes(self, aliases):
        """Загрузка атрибутов узлов."""
        if not aliases:
            return pd.DataFrame()
        query = f"""
            SELECT alias, label, degree, total_received
            FROM node_attributes
            WHERE alias IN ({','.join(map(str, aliases))})
        """
        logger.debug(f"Загрузка атрибутов узлов: {str(query)}")
        try:
            df = pd.read_sql(query, self.conn)
            logger.info(f"Загружено {len(df)} атрибутов узлов.")
            return df
        except Exception as e:
            logger.error(f"Ошибка при загрузке атрибутов узлов: {e}")
            return pd.DataFrame()

    def _load_edges(self, aliases):
        """Загрузка ребер."""
        if not aliases:
            return pd.DataFrame()
        query = f"""
            SELECT a, b, total_sent
            FROM edge_attributes
            WHERE a IN ({','.join(map(str, aliases))})
               OR b IN ({','.join(map(str, aliases))})
        """
        logger.debug(f"Загрузка ребер: {str(query)}")
        try:
            df = pd.read_sql(query, self.conn)
            logger.info(f"Загружено {len(df)} рёбер.")
            return df
        except Exception as e:
            logger.error(f"Ошибка при загрузке рёбер: {e}")
            return pd.DataFrame()

    def _construct_graph_data(self, nodes_df, edges_df):
        """Построение графа из данных."""
        alias_to_idx = {alias: i for i, alias in enumerate(nodes_df['alias'])}
        edges, edge_attrs = self._process_edges(edges_df, alias_to_idx)
        edge_index = torch.tensor(edges, dtype=torch.long).T if edges else torch.zeros((2, 0), dtype=torch.long)
        edge_attr = torch.tensor(edge_attrs, dtype=torch.float).view(-1, 1) if edge_attrs else torch.zeros((0, 1), dtype=torch.float)
        x = torch.tensor(nodes_df[['degree', 'total_received']].fillna(0).values, dtype=torch.float)
        y = torch.tensor(nodes_df['label'].fillna(0).values, dtype=torch.long)
        return Data(x=x, edge_index=edge_index, edge_attr=edge_attr, y=y)

    def _process_edges(self, edges_df, alias_to_idx):
        """Обработка ребер и их атрибутов."""
        edges = []
        edge_attrs = []
        for _, row in edges_df.iterrows():
            a_idx = alias_to_idx.get(row['a'])
            b_idx = alias_to_idx.get(row['b'])
            if a_idx is not None and b_idx is not None:
                edges.append([a_idx, b_idx])
                edge_attrs.append(row['total_sent'])
        return edges, edge_attrs

    def _empty_data(self):
        """Возвращает пустой граф."""
        logger.warning("Возвращён пустой граф.")
        return Data(
            x=torch.tensor([], dtype=torch.float),
            edge_index=torch.tensor([], dtype=torch.long).view(2, -1),
            edge_attr=torch.tensor([], dtype=torch.float).view(-1, 1),
            y=torch.tensor([], dtype=torch.long)
        )



class TestBitcoinGraphDataset(unittest.TestCase):
    """Тесты для BitcoinGraphDataset."""

    def setUp(self):
        """Подготовка тестового окружения."""
        self.db_config = {
            'dbname': 'test_db',
            'user': 'test_user',
            'password': 'test_pass',
            'host': 'localhost',
            'port': 5432
        }

        self.nodes_data = pd.DataFrame({
            'alias': [1, 2, 3, 4],
            'label': [0, 1, 0, 1],
            'degree': [2, 3, 1, 2],
            'total_received': [100.0, 200.0, 150.0, 300.0]
        })

        self.edges_data = pd.DataFrame({
            'a': [1, 2, 3, 1, 4],
            'b': [2, 3, 4, 4, 1],
            'total_sent': [50.0, 30.0, 20.0, 40.0, 60.0]
        })

    def _setup_mock(self, mock_connect, total_nodes=4, nodes_data=None):
        """Настройка моков для тестов с правильными типами psycopg2."""
        mock_conn = MagicMock(spec=psycopg2.extensions.connection)
        mock_cursor = MagicMock(spec=psycopg2.extensions.cursor)

        mock_cursor.fetchone.return_value = [total_nodes]
        mock_conn.cursor.return_value = mock_cursor
        mock_connect.return_value = mock_conn

        return mock_conn, mock_cursor, nodes_data if nodes_data is not None else self.nodes_data

    @patch('psycopg2.connect')
    @patch('pandas.read_sql')
    def test_neighbor_sampling(self, mock_read_sql, mock_connect):
        """Тест семплирования окрестностей."""
        mock_conn, mock_cursor, nodes_data = self._setup_mock(mock_connect, 4)

        def read_sql_side_effect(query, *args):
            if 'LIMIT' in query:
                return self.nodes_data.head(2)
            elif 'WHERE alias IN' in query:
                return self.nodes_data
            elif 'SELECT a, b' in query:
                return self.edges_data
            return pd.DataFrame()

        mock_read_sql.side_effect = read_sql_side_effect

        with patch.object(mock_cursor, 'execute') as mock_execute:
            with patch.object(mock_cursor, 'fetchall', return_value=[(1,), (2,), (3,), (4,)]):
                dataset = BitcoinGraphDataset(self.db_config, batch_size=2, num_hops=2, num_neighbors=3)
                batch = dataset[0]

                self.assertEqual(batch.x.shape[0], 4)
                self.assertEqual(batch.edge_index.shape[1], 5)
                self.assertEqual(batch.y.shape[0], 4)

    @patch('psycopg2.connect')
    @patch('pandas.read_sql')
    def test_minimal_neighborhood(self, mock_read_sql, mock_connect):
        """Тест минимальной окрестности."""
        # Создаем моки с правильными типами
        mock_conn = MagicMock(spec=psycopg2.extensions.connection)
        mock_cursor = MagicMock(spec=psycopg2.extensions.cursor)

        mock_conn.cursor.return_value = mock_cursor
        mock_connect.return_value = mock_conn

        minimal_nodes = pd.DataFrame({
            'alias': [1],
            'label': [0],
            'degree': [1],
            'total_received': [100.0]
        })

        minimal_edges = pd.DataFrame({
            'a': [1],
            'b': [1],
            'total_sent': [50.0]
        })

        def read_sql_side_effect(query, *args):
            if 'LIMIT' in query:
                return minimal_nodes
            elif 'WHERE alias IN' in query:
                return minimal_nodes
            elif 'SELECT a, b' in query:
                return minimal_edges
            return pd.DataFrame()

        mock_read_sql.side_effect = read_sql_side_effect

        with patch.object(mock_cursor, 'execute'), \
            patch.object(mock_cursor, 'fetchall', return_value=[(1,)]):
            dataset = BitcoinGraphDataset(self.db_config, batch_size=1, num_hops=1, num_neighbors=1)
            batch = dataset[0]

            self.assertEqual(batch.x.shape, (1, 2))
            self.assertEqual(batch.edge_index.shape, (2, 1))
            self.assertEqual(batch.edge_attr.shape, (1, 1))
            self.assertEqual(batch.y.shape, (1,))

    @patch('psycopg2.connect')
    @patch('pandas.read_sql')
    def test_getitem_returns_correct_data(self, mock_read_sql, mock_connect):
        """Тест __getitem__ возвращает корректные данные."""
        test_nodes = self.nodes_data.head(2)  # Алиасы: 1, 2
        mock_conn, mock_cursor, _ = self._setup_mock(mock_connect, 2)

        def mock_sql_side_effect(query, *args, **kwargs):
            if 'LIMIT' in query and 'alias' in query:
                return test_nodes
            elif 'WHERE alias IN' in query:
                return test_nodes
            elif 'SELECT a, b' in query:
                # Возвращаем только те рёбра, где оба узла есть в test_nodes
                edges_subset = self.edges_data[
                    (self.edges_data['a'].isin(test_nodes['alias'])) &
                    (self.edges_data['b'].isin(test_nodes['alias']))
                ]
                # Добавляем недостающее ребро, если нужно, чтобы было 2 ребра
                if len(edges_subset) < 2:
                    new_rows = pd.DataFrame({
                        'a': [2],
                        'b': [1],
                        'total_sent': [30.0]
                    })
                    edges_subset = pd.concat([edges_subset, new_rows], ignore_index=True)
                return edges_subset
            return pd.DataFrame()

        mock_read_sql.side_effect = mock_sql_side_effect

        with patch.object(mock_cursor, 'execute'), \
            patch.object(mock_cursor, 'fetchall', return_value=[(1,), (2,)]):
            dataset = BitcoinGraphDataset(
                self.db_config,
                batch_size=2,
                num_hops=1,
                num_neighbors=2
            )
            batch = dataset[0]

        self.assertIsInstance(batch, Data)
        self.assertEqual(batch.x.shape, (2, 2))
        self.assertEqual(batch.edge_index.shape, (2, 2))  # Теперь точно 2 ребра
        self.assertEqual(batch.edge_attr.shape, (2, 1))
        self.assertEqual(batch.y.shape, (2,))

    @patch('psycopg2.connect')
    @patch('pandas.read_sql')
    def test_sampling_errors(self, mock_read_sql, mock_connect):
        """Тест обработки ошибок при семплировании окрестностей."""
        # Подавляем логи внутри теста
        logger = logging.getLogger('__main__')
        logger.setLevel(logging.CRITICAL)  # Отключаем вывод ERROR и WARNING

        # Подготавливаем моки для подключения к БД
        mock_conn, mock_cursor, nodes_data = self._setup_mock(mock_connect, 2)

        # Настраиваем side_effect для read_sql: возвращаем данные для _load_batch_nodes и _load_node_attributes
        def read_sql_side_effect(query, *args):
            if 'LIMIT' in query and 'alias' in query:
                return self.nodes_data.head(1)  # Возвращаем минимум 1 узел
            elif 'WHERE alias IN' in query:
                return self.nodes_data.head(1)
            return pd.DataFrame()

        mock_read_sql.side_effect = read_sql_side_effect

        # Настраиваем execute так, чтобы только рекурсивный запрос выбрасывал ошибку
        def execute_side_effect(query, *args):
            if 'WITH RECURSIVE' in query:
                raise Exception("Sampling error")  # Только для семплирования окрестностей
            else:
                return None  # Остальные запросы выполняются успешно

        with patch.object(mock_cursor, 'execute') as mock_execute:
            mock_execute.side_effect = execute_side_effect
            dataset = BitcoinGraphDataset(self.db_config, num_hops=2, num_neighbors=2)
            batch = dataset[0]  # Вызываем __getitem__

            # Проверяем, что в случае ошибки возвращается пустой граф
            self.assertTrue(batch.x.numel() == 0, "Node features tensor should be empty")
            self.assertTrue(batch.edge_index.numel() == 0, "Edge index tensor should be empty")
            self.assertTrue(batch.edge_attr.numel() == 0, "Edge attribute tensor should be empty")
            self.assertTrue(batch.y.numel() == 0, "Node labels tensor should be empty")


    @patch('psycopg2.connect')
    @patch('pandas.read_sql')
    def test_sql_query_format(self, mock_read_sql, mock_connect):
        """Тест корректности формирования SQL-запросов.

        Проверяет, что:
        1. SQL-запрос семплирования окрестностей содержит правильные
          значения параметров num_hops и num_neighbors
        2. Подстановка параметров в рекурсивный запрос выполняется корректно
        """
        # Тестовые параметры
        test_num_hops = 3
        test_num_neighbors = 2

        # Подготавливаем моки для подключения к БД
        mock_conn, mock_cursor, nodes_data = self._setup_mock(mock_connect, 2)

        # Настраиваем side_effect для проверки SQL-запроса
        def side_effect(query, *args, **kwargs):
            """Проверяем содержание SQL-запроса при семплировании окрестностей."""
            if 'WITH RECURSIVE' in query:
                # Проверяем корректность подстановки параметров в SQL
                self.assertIn(
                    f"hop < {test_num_hops}",
                    query,
                    "Запрос должен содержать условие hop < num_hops"
                )
                self.assertIn(
                    f"rn <= {test_num_neighbors}",
                    query,
                    "Запрос должен содержать ограничение rn <= num_neighbors"
                )
                return pd.DataFrame()  # Для семплирования — пустой результат

            elif 'LIMIT' in query and 'alias' in query:
                # Возвращаем тестовые данные для _load_batch_nodes
                return self.nodes_data.head(1)  # Минимум 1 узел

            elif 'WHERE alias IN' in query:
                return self.nodes_data.head(1)  # Атрибуты этого узла

            elif 'SELECT a, b' in query:
                return pd.DataFrame()  # Можно вернуть пустые рёбра

            return pd.DataFrame()  # Все остальные случаи

        # Привязываем side_effect к моку
        mock_read_sql.side_effect = side_effect

        with patch.object(mock_cursor, 'execute'), \
            patch.object(mock_cursor, 'fetchall', return_value=[(1,)]):
            # Создаем экземпляр датасета с тестовыми параметрами
            dataset = BitcoinGraphDataset(
                self.db_config,
                num_hops=test_num_hops,
                num_neighbors=test_num_neighbors
            )

            # Вызываем __getitem__ (что запустит выполнение SQL-запросов)
            batch = dataset[0]

            # Проверяем, что batch существует
            self.assertIsInstance(batch, Data)

    @patch('psycopg2.connect')
    @patch('pandas.read_sql')
    def test_real_data_sampling(self, mock_read_sql, mock_connect):
        """Тест обработки реальных данных с семплированием окрестностей."""
        # Подготавливаем моки для подключения к БД
        mock_conn, mock_cursor, nodes_data = self._setup_mock(mock_connect, 4)

        # Настраиваем side_effect для имитации SQL-запросов
        def mock_sql_side_effect(query, *args, **kwargs):
            """Обрабатывает SQL-запросы и возвращает тестовые данные."""
            if 'LIMIT' in query and 'alias' in query:  # Загрузка батча узлов
                return nodes_data.head(2)  # Первые 2 узла (алиасы 1 и 2)
            elif 'IN (' in query and 'alias' in query:  # Загрузка атрибутов всех алиасов
                return nodes_data  # Возвращаем все 4 узла
            elif 'SELECT a, b' in query:  # Загрузка ребер
                return self.edges_data  # Используем тестовые ребра
            return pd.DataFrame()  # Все остальные запросы возвращают пустой DataFrame

        # Привязываем side_effect к моку
        mock_read_sql.side_effect = mock_sql_side_effect

        # Настраиваем возврат алиасов из рекурсивного SQL-запроса
        with patch.object(mock_cursor, 'execute') as mock_execute:
            with patch.object(mock_cursor, 'fetchall', return_value=[(1,), (2,), (3,), (4,)]):
                # Создаем экземпляр датасета с тестовыми параметрами
                dataset = BitcoinGraphDataset(
                    self.db_config,
                    batch_size=2,
                    num_hops=2,
                    num_neighbors=3
                )
                # Получаем первый батч данных
                batch = dataset[0]

        # Проверяем структуру графа
        self.assertEqual(
            len(batch.x), 4,
            "Количество узлов в графе должно быть равно 4"
        )
        self.assertEqual(
            len(batch.edge_index[0]), 5,
            "Количество ребер в графе должно быть равно 5"
        )
        # Проверяем совместимость с GCN-слоем
        class DummyGCN(torch.nn.Module):
            """Простая GCN-модель для тестирования."""
            def __init__(self):
                super().__init__()
                self.conv = GCNConv(2, 2)  # 2 входных признака, 2 выходных
            def forward(self, x, edge_index):
                return self.conv(x, edge_index)
        # Создаем экземпляр модели
        model = DummyGCN()
        # Выполняем прямой проход без вычисления градиентов
        with torch.no_grad():
            out = model(batch.x, batch.edge_index)
        # Проверяем размерность выхода модели
        self.assertEqual(
            out.shape, (4, 2),
            "Размерность выхода модели должна быть (4, 2)"
        )

    @patch('psycopg2.connect')
    @patch('pandas.read_sql')
    def test_getitem_returns_empty_graph_when_no_edges(self, mock_read_sql, mock_connect):
        """Тест обработки случая, когда в батче нет рёбер.
        Проверяет, что датасет корректно обрабатывает отсутствие рёбер
        и возвращает граф с пустыми edge_index и edge_attr.
        """
        # Подготавливаем моки для подключения к БД
        mock_conn, mock_cursor, nodes_data = self._setup_mock(mock_connect, 2)

        def side_effect(query, *args, **kwargs):
            """Обрабатывает SQL-запросы и возвращает тестовые данные."""
            if 'LIMIT' in query and 'alias' in query:  # Загрузка батча узлов
                return nodes_data.head(2)  # Возвращаем 2 узла (алиасы 1 и 2)
            elif 'WHERE alias IN' in query:  # Загрузка атрибутов всех алиасов
                return nodes_data.head(2)  # Используем те же 2 узла
            elif 'SELECT a, b' in query:  # Загрузка рёбер (возвращаем пустой DataFrame)
                return pd.DataFrame(columns=['a', 'b', 'total_sent'])
            return pd.DataFrame()  # Все остальные запросы возвращают пустой DataFrame

        # Привязываем side_effect к моку
        mock_read_sql.side_effect = side_effect

        with patch.object(mock_cursor, 'execute'), \
            patch.object(mock_cursor, 'fetchall', return_value=[(1,), (2,)]):
            # Создаем экземпляр датасета с тестовыми параметрами
            dataset = BitcoinGraphDataset(
                self.db_config,
                batch_size=2,
                num_hops=1,
                num_neighbors=2
            )
            # Получаем первый батч данных
            batch = dataset[0]

        # Проверяем структуру графа
        self.assertIsInstance(batch, Data, "Батч должен быть экземпляром torch_geometric.data.Data")
        self.assertEqual(
            batch.x.shape, (2, 2),
            "Размерность признаков узлов должна быть (2, 2)"
        )
        self.assertEqual(
            batch.edge_index.shape, (2, 0),
            "Размерность edge_index должна быть (2, 0) для пустых рёбер"
        )
        self.assertEqual(
            batch.edge_attr.shape, (0, 1),
            "Размерность edge_attr должна быть (0, 1) для пустых рёбер"
        )
        self.assertEqual(
            batch.y.shape, (2,),
            "Размерность меток должна быть (2,)"
        )

    @patch('psycopg2.connect')
    def test_len_returns_correct_value(self, mock_connect):
        """Тест корректного вычисления длины датасета.

        Проверяет, что метод __len__ возвращает правильное количество батчей:
        - Учитывает остаток от деления (округление вверх)
        - Работает при различных значениях batch_size и total_nodes
        """
        # Подготавливаем моки для подключения к БД
        mock_conn, mock_cursor, nodes_data = self._setup_mock(mock_connect, 65)

        # Тестовый случай: 65 узлов, batch_size=32
        # Ожидаемое значение: (65 + 32 - 1) // 32 = 96 // 32 = 3
        dataset = BitcoinGraphDataset(self.db_config, batch_size=32)
        self.assertEqual(
            len(dataset), 3,
            "Длина датасета должна быть равна 3 для 65 узлов и batch_size=32"
        )

        # Дополнительные проверки для других значений
        # Тестовый случай: 64 узла, batch_size=32 (точное деление)
        mock_conn, mock_cursor, nodes_data = self._setup_mock(mock_connect, 64)
        dataset = BitcoinGraphDataset(self.db_config, batch_size=32)
        self.assertEqual(
            len(dataset), 2,
            "Длина датасета должна быть равна 2 для 64 узлов и batch_size=32"
        )

        # Тестовый случай: 1 узел, batch_size=32
        mock_conn, mock_cursor, nodes_data = self._setup_mock(mock_connect, 1)
        dataset = BitcoinGraphDataset(self.db_config, batch_size=32)
        self.assertEqual(
            len(dataset), 1,
            "Длина датасета должна быть равна 1 для 1 узла и batch_size=32"
        )

        # Тестовый случай: 0 узлов (не поддерживается в _setup_mock, но проверяем)
        mock_conn, mock_cursor, nodes_data = self._setup_mock(mock_connect, 0)
        dataset = BitcoinGraphDataset(self.db_config, batch_size=32)
        self.assertEqual(
            len(dataset), 0,
            "Длина датасета должна быть равна 0 для 0 узлов"
        )

    @patch('psycopg2.connect')
    @patch('pandas.read_sql')
    def test_model_compatibility(self, mock_read_sql, mock_connect):
        """Тест совместимости датасета с GCN-моделью."""
        # Подготавливаем моки для подключения к БД
        mock_conn, mock_cursor, nodes_data = self._setup_mock(mock_connect, 2)
        # Явно определяем test_nodes
        test_nodes = self.nodes_data.head(2)  # Первые 2 узла (алиасы 1 и 2)

        def side_effect(query, *args, **kwargs):
            """Обрабатывает SQL-запросы и возвращает тестовые данные."""
            if 'LIMIT' in query and 'alias' in query:  # Загрузка батча узлов
                return test_nodes
            elif 'WHERE alias IN' in query:  # Загрузка атрибутов узлов по алиасам
                return test_nodes  # Используем те же 2 узла
            elif 'WITH RECURSIVE' in query:  # Семплирование окрестностей
                return test_nodes  # Используем те же 2 узла
            elif 'SELECT a, b' in query:  # Загрузка рёбер
                # Берем первые 2 ребра и корректируем их, чтобы они соответствовали test_nodes
                edges_subset = self.edges_data.head(2).copy()
                edges_subset['a'] = edges_subset['a'].apply(lambda x: x if x in [1,2] else 1)
                edges_subset['b'] = edges_subset['b'].apply(lambda x: x if x in [1,2] else 2)
                new_row = pd.DataFrame({'a': [2], 'b': [1], 'total_sent': [30.0]})
                edges_subset = pd.concat([edges_subset, new_row], ignore_index=True)
                return edges_subset
            return pd.DataFrame()  # Все остальные запросы возвращают пустой DataFrame

        # Привязываем side_effect к моку
        mock_read_sql.side_effect = side_effect

        with patch.object(mock_cursor, 'execute'), \
            patch.object(mock_cursor, 'fetchall', return_value=[(1,), (2,)]):
            # Создаем экземпляр датасета с тестовыми параметрами
            dataset = BitcoinGraphDataset(
                self.db_config,
                batch_size=2,
                num_hops=1,
                num_neighbors=2
            )
            # Получаем первый батч данных
            batch = dataset[0]

        # Проверяем структуру графа
        self.assertIsInstance(batch, Data, "Батч должен быть экземпляром torch_geometric.data.Data")
        self.assertEqual(
            batch.x.shape, (2, 2),
            "Размерность признаков узлов должна быть (2, 2)"
        )


In [3]:
if __name__ == '__main__':
    unittest.main(argv=[''], exit=False)

.........
----------------------------------------------------------------------
Ran 9 tests in 0.260s

OK
